In [1]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import pymongo
import streamlit as st
from plotly.subplots import make_subplots


## Load Data

In [2]:
Client = pymongo.MongoClient("mongodb://localhost:27017/")
db = Client["streaming_data"]
collection = db["5m_stock_datastream"]
# Fetch NVDA and PLTR symbol from the database
nvda_df = pd.DataFrame(list(collection.find({'symbol':'NVDA'})))
pltr_df = pd.DataFrame(list(collection.find({'symbol':'PLTR'})))

df = pd.concat([nvda_df, pltr_df], axis=0)

In [3]:
df.head()

,date,symbol,volume,datetime,low,close,open,high,_id
0,2024-09-30 09:30:00,NVDA,19666146,2024-09-30 09:30:00,118.660004,119.610001,118.750000,119.619102,66faeb62fa82d5e0c222ccdf
1,2024-09-30 09:35:00,NVDA,7979334,2024-09-30 09:35:00,119.500000,120.889801,119.725403,120.989998,66faeb62fa82d5e0c222cce0
2,2024-09-30 09:40:00,NVDA,5265461,2024-09-30 09:40:00,120.260002,120.959999,120.879997,121.099998,66faeb62fa82d5e0c222cce2
3,2024-09-30 09:45:00,NVDA,4612635,2024-09-30 09:45:00,120.680000,120.949997,120.838997,121.269997,66faeb62fa82d5e0c222cce4
4,2024-09-30 09:50:00,NVDA,3396717,2024-09-30 09:50:00,120.449997,120.459999,120.959999,121.139999,66faeb62fa82d5e0c222cce5


## Support Resistance Detection

#### Fratal Candles Patterns
##### Credit : https://towardsdatascience.com/detection-of-price-support-and-resistance-levels-in-python-baedc44c34c9
Definition: 

A fractal is a candlestick pattern made by 5 candles. The third candle has the lowest low price, the previous candles have decreasing lows and the next candles have increasing lows. By this pattern, the low of the third candle is the support level. The same concept applies to resistance levels, where the third candle has the highest high of the five ones.

![Fractals Pattern](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*xOLF6WDrcvVks7CEMIVoGw.png)

### Compute Support and Resistance

* Rolling 5 days and compute the low and high
* Center Rolling Required

In [4]:
# Define the support and resistance levels
batch = {}
window = 15
alert_df = pd.DataFrame()

def store_batch_alert(symbol, alert):
    global alert_df
    # Dummy implementation
    # Store the alert in a DataFrame
    alert_df = pd.concat([alert_df, pd.DataFrame([alert])], ignore_index=True)
    
class trend_pattern:
    def __init__(self, lookback, batch_data):
        self.lookback = lookback
        self.batch_data = batch_data
    
    def strong_support(self):
        # Dummy implementation
        support_line =  self.batch_data['low'].rolling(5, center=True).min()
        support_df = self.batch_data[self.batch_data['low'] == support_line]
        return support_df
    
    def strong_resistance(self):
        # Dummy implementation
        resistance_line = self.batch_data['high'].rolling(5, center=True).max()
        resistance_df = self.batch_data[self.batch_data['high'] == resistance_line]
        return resistance_df

for records in df.to_dict(orient='records'):
    symbol = records['symbol']
    if symbol not in batch:
        batch[symbol] = []

    batch[symbol].append(records)

    # Check if we have enough data for batch processing
    if len(batch[symbol]) >= window:
        filter_df = pd.DataFrame(batch[symbol])
        # Extract the strong support and resistance
        support = trend_pattern(lookback=window, batch_data=filter_df).strong_support()
        resistance = trend_pattern(lookback=window, batch_data=filter_df).strong_resistance()
        
        # Store Alert to MongoDB
        if len(support) != 0:
            for i in support.itertuples():
                store_batch_alert(symbol, {'symbol': symbol, 'support': i.low, 'datetime': i.datetime})    
        if len(resistance) != 0:
            for i in resistance.itertuples():
                store_batch_alert(symbol, {'symbol': symbol, 'resistance': i.high, 'datetime': i.datetime})
        
        # Clear the batch for the symbol after processing
        
        batch[symbol] = []

### Visaulzie the Support and Resistance

In [5]:
# Select the data for the specific symbol (example: 'NVDA')
symbol = 'NVDA'
select_df = df[df['symbol'] == symbol]

# Create subplots: 2 rows, 1 column
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.1, 
                    row_heights=[0.7, 0.3],
                    subplot_titles=('Candlestick Chart', 'Support and Resistance Histogram'))

# Add candlestick chart to the first subplot
fig.add_trace(go.Candlestick(x=select_df['datetime'].astype(str),  # Convert to string for categorical x-axis
                            open=select_df['open'],
                            high=select_df['high'],
                            low=select_df['low'],
                            close=select_df['close'],
                            increasing_line_color='green',  # Color for bullish (increasing) candles
                            decreasing_line_color='red'  # Color for bearish (decreasing) candles
                        ), row=1, col=1)

# Add support and resistance lines to the candlestick chart
if 'alert_df' in globals():
    # Ensure alert_df is filtered for the specific symbol
    alert_df_symbol = alert_df[alert_df['symbol'] == symbol]
    
    # Add support lines
    if 'support' in alert_df_symbol.columns:
        for _, row in alert_df_symbol.iterrows():
            end_date = row['datetime'] + pd.Timedelta(minutes=60)
            row['datetime'] = row['datetime'].strftime('%Y-%m-%d %H:%M:%S')
            end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
            fig.add_trace(go.Scatter(x=[row['datetime'], end_date], 
                                    y=[row['support'], row['support']], 
                                    mode='lines', 
                                    line=dict(color='blue', width=1, dash='dash'), 
                                    name='Support'), row=1, col=1)

    # Add resistance lines
    if 'resistance' in alert_df_symbol.columns:
        for _, row in alert_df_symbol.iterrows():
            end_date = row['datetime'] + pd.Timedelta(minutes=60)
            row['datetime'] = row['datetime'].strftime('%Y-%m-%d %H:%M:%S')
            end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
            fig.add_trace(go.Scatter(x=[row['datetime'], end_date], 
                                    y=[row['resistance'], row['resistance']], 
                                    mode='lines', 
                                    line=dict(color='red', width=1, dash='dash'), 
                                    name='Resistance'), row=1, col=1)

# Create histogram data for support and resistance
support_histogram = alert_df_symbol['support'].value_counts().reset_index()
support_histogram.columns = ['price', 'count']

resistance_histogram = alert_df_symbol['resistance'].value_counts().reset_index()
resistance_histogram.columns = ['price', 'count']

# Add histogram for support levels to the second subplot
fig.add_trace(go.Bar(x=support_histogram['price'], 
                    y=support_histogram['count'], 
                    name='Support', 
                    marker_color='blue'), row=2, col=1)

# Add histogram for resistance levels to the second subplot
fig.add_trace(go.Bar(x=resistance_histogram['price'], 
                    y=resistance_histogram['count'], 
                    name='Resistance', 
                    marker_color='red'), row=2, col=1)

# Update layout
fig.update_layout(
    title=f'Candlestick chart and Support/Resistance Histogram for {symbol}',
    xaxis_rangeslider_visible=False,  # Hide range slider
    showlegend=False,  # Hide legend    
    xaxis_type='category'  # Set x-axis type to categorical to remove gaps
)

# Show the figure
fig.show()

## Engulfing Method

In [6]:
# Generate random test data
np.random.seed(42)  # For reproducibility

# Define the number of rows for the test DataFrame
num_rows = 100

# Generate random dates
date_rng = pd.date_range(start='2024-09-27 09:30:00', periods=num_rows, freq='5T')

# Generate random data for the DataFrame
test_data = {
    'date': date_rng,
    'symbol': np.random.choice(['AAPL', 'GOOGL', 'MSFT', 'NVDA', 'TSLA'], size=num_rows),
    'interval': np.random.choice(['5m','15m','30m'], size=num_rows),
    '_id': [f'id_{i}' for i in range(num_rows)],
    'datetime': date_rng
}

# Generate random walk for OHLC data
initial_price = 150
price_changes = np.random.normal(loc=0, scale=1, size=num_rows)
prices = initial_price + np.cumsum(price_changes)

test_data['open'] = prices
test_data['high'] = prices + np.random.uniform(low=0, high=5, size=num_rows)
test_data['low'] = prices - np.random.uniform(low=0, high=5, size=num_rows)
test_data['close'] = prices + np.random.uniform(low=-2, high=2, size=num_rows)

# Convert to DataFrame and sort
test_data = pd.DataFrame(test_data).sort_values(by=['symbol','interval','date'])

# Display the test DataFrame
test_data

/var/folders/pv/rbjx7k1j5cs6jxs5pskztclh0000gn/T/ipykernel_72139/2835045034.py:8: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



,date,symbol,interval,_id,datetime,open,high,low,close
23,2024-09-27 11:25:00,AAPL,15m,id_23,2024-09-27 11:25:00,152.582965,155.239738,148.070201,152.727351
85,2024-09-27 16:35:00,AAPL,15m,id_85,2024-09-27 16:35:00,155.754880,158.482964,152.838036,153.813059
91,2024-09-27 17:05:00,AAPL,15m,id_91,2024-09-27 17:05:00,157.725543,158.072350,152.909431,157.118565
24,2024-09-27 11:30:00,AAPL,30m,id_24,2024-09-27 11:30:00,151.799712,154.502888,149.273450,151.037822
33,2024-09-27 12:15:00,AAPL,30m,id_33,2024-09-27 12:15:00,152.213762,152.606044,147.463453,151.886735
...,...,...,...,...,...,...,...,...,...
62,2024-09-27 14:40:00,TSLA,5m,id_62,2024-09-27 14:40:00,154.089648,156.547728,153.665460,155.905852
71,2024-09-27 15:25:00,TSLA,5m,id_71,2024-09-27 15:25:00,158.018970,161.148269,157.601466,159.108243
72,2024-09-27 15:30:00,TSLA,5m,id_72,2024-09-27 15:30:00,157.795507,160.311188,153.909772,157.876161
80,2024-09-27 16:10:00,TSLA,5m,id_80,2024-09-27 16:10:00,157.172253,161.873404,156.890736,158.192802


In [7]:
import pandas as pd

class CandlePattern:
    def __init__(self, candle: pd.DataFrame):
        self.candle = candle
        self.body_size = abs(self.candle['close'] - self.candle['open'])
        self.lower_shadow = min(self.candle['open'], self.candle['close']) - self.candle['low']
        self.upper_shadow = self.candle['high'] - max(self.candle['open'], self.candle['close']) 
    
    def no_volume(self):
        return self.candle['close'] == self.candle['open'] == self.candle['high'] == self.candle['low']
    
    def hammer_alert(self):
        # Hammer candle pattern: small body, long lower shadow, little or no upper shadow
        is_hammer = (
            (self.body_size <= (self.candle['high'] - self.candle['low']) * 0.3) &  # Small body
            (self.lower_shadow >= (self.candle['high'] - self.candle['low']) * 0.7) &  # Long lower shadow
            (self.upper_shadow <= (self.candle['high'] - self.candle['low']) * 0.1)  # Little or no upper shadow
        )
        
        date = pd.to_datetime(self.candle['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        return is_hammer, date

    def engulf_alert(self, prev_candle: pd.DataFrame, current_candle: pd.DataFrame):
        # Previous Candle
        prev_open = prev_candle['open']
        prev_close = prev_candle['close']

        # Current Candle
        current_open = current_candle['open']
        current_close = current_candle['close']
        current_high = current_candle['high']
        
        # Bullish Engulfing: current green candle engulfs previous candle
        bullish_engulfing = (
            (current_close > current_open) &  # Current candle is green
            (current_open < min(prev_open, prev_close)) & 
            (current_close > max(prev_open, prev_close)) &
            ((current_close - current_open) > abs(prev_close - prev_open) * 1.5) &  # Current candle significantly larger
            ((current_high - current_close) * 0.7 <= (current_close - current_open)) # Little or no upper shadow
        )
        
        # Bearish Engulfing: current red candle engulfs previous candle
        bearish_engulfing = (
            (current_close < current_open) &  # Current candle is red
            (current_open > max(prev_open, prev_close)) & 
            (current_close < min(prev_open, prev_close))
        )

        date = pd.to_datetime(current_candle['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        
        return bullish_engulfing, bearish_engulfing, date



In [8]:
candle = {}
last_processed_index = {}
pointer_a = 0
pointer_b = 1
alert_data = {}
last_processed_interval = 0

def streaming_process(data, interval):
    global candle, last_processed_index, pointer_a, pointer_b, alert_data, last_processed_interval

    # Store the processed index in a dict
    if data['symbol'] not in last_processed_index:
        last_processed_index[data['symbol']] = -1
        last_processed_interval = interval 
    
    # Store the candle data in a dict
    if data['symbol'] not in candle:
        candle[data['symbol']] = []
        
    candle[data['symbol']].append(data)

    # Initialize alert_data for the symbol if it does not exist
    if data['symbol'] not in alert_data:
        alert_data[data['symbol']] = []

    # If interval changes, reset the last processed index
    if last_processed_interval != interval:
        last_processed_interval = interval
        pointer_a = 0
        pointer_b = 1
        for symbol in last_processed_index:
            last_processed_index[symbol] = -1
            
    # Loop through the candle data from the last processed index
    for i in range(last_processed_index[data['symbol']] + 1, len(candle[data['symbol']])):
        if last_processed_index[data['symbol']] == -1:
            pointer_a = 0
            pointer_b = 1
            curr_candle = candle[data['symbol']][pointer_a]
            candle_pattern = CandlePattern(curr_candle)
            bullish_382, bullish_382_date = candle_pattern.hammer_alert()
            if bullish_382:
                alert_data[data['symbol']].append({'date': bullish_382_date, 'alert': 'bullish_382'})
                
            print(f"symbol: {data['symbol']} analyzing {curr_candle['datetime']} | Processed Index: {last_processed_index[data['symbol']]} | Interval: {last_processed_interval}")
            print(f"current candle: {curr_candle['datetime']}")
            
        curr_candle = candle[data['symbol']][i]
        candle_pattern = CandlePattern(curr_candle)
        # Check if there are enough candles to compare
        if len(candle[data['symbol']]) > 1 and pointer_b < len(candle[data['symbol']]):
            pre_candle = candle[data['symbol']][pointer_a]
            curr_candle = candle[data['symbol']][pointer_b]
            
            # Check for bullish 0.382 candle patterns
            bullish_382, bullish_382_date = candle_pattern.hammer_alert()

            # Check for engulfing patterns
            bullish_engulfing, bearish_engulfing, date = candle_pattern.engulf_alert(pre_candle, curr_candle)

            if bullish_engulfing:
                alert_data[data['symbol']].append({'date': date, 'alert': 'bullish_engulfing'})
                                
            # Update the last processed index
            last_processed_index[data['symbol']] = pointer_b
            print(f"symbol: {data['symbol']} comparing {pre_candle['datetime']} and {curr_candle['datetime']} | Processed Index: {last_processed_index[data['symbol']]} | Interval: {last_processed_interval}")
            print(f"pointer a: {pointer_a} | pointer b: {pointer_b}")
            print(f"current candle: {curr_candle['datetime']}")

        
        pointer_a += 1
        pointer_b += 1                

        break
    
# Test the function with the generated test data
for interval in test_data['interval'].unique():
    for index, row in test_data.iterrows():
        streaming_process(row, interval)

symbol: AAPL analyzing 2024-09-27 11:25:00 | Processed Index: -1 | Interval: 15m
current candle: 2024-09-27 11:25:00
symbol: AAPL analyzing 2024-09-27 11:25:00 | Processed Index: -1 | Interval: 15m
current candle: 2024-09-27 11:25:00
symbol: AAPL comparing 2024-09-27 11:25:00 and 2024-09-27 16:35:00 | Processed Index: 1 | Interval: 15m
pointer a: 0 | pointer b: 1
current candle: 2024-09-27 16:35:00
symbol: AAPL comparing 2024-09-27 16:35:00 and 2024-09-27 17:05:00 | Processed Index: 2 | Interval: 15m
pointer a: 1 | pointer b: 2
current candle: 2024-09-27 17:05:00
symbol: AAPL comparing 2024-09-27 17:05:00 and 2024-09-27 11:30:00 | Processed Index: 3 | Interval: 15m
pointer a: 2 | pointer b: 3
current candle: 2024-09-27 11:30:00
symbol: AAPL comparing 2024-09-27 11:30:00 and 2024-09-27 12:15:00 | Processed Index: 4 | Interval: 15m
pointer a: 3 | pointer b: 4
current candle: 2024-09-27 12:15:00
symbol: AAPL comparing 2024-09-27 12:15:00 and 2024-09-27 15:50:00 | Processed Index: 5 | Inte

In [9]:
# Fetch data from mongodb 
import pymongo
from pymongo import MongoClient
import pandas as pd

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['streaming_data']
collection = db['5m_stock_datastream']

# Fetch data from MongoDB
data = collection.find({'symbol': 'BSX'})
df = pd.DataFrame(list(data))
df = df.drop(columns=['_id'])
df = df[df['datetime'] >= '2024-10-01 09:30:00']
df.head()

,date,symbol,volume,datetime,low,close,open,high
78,2024-10-01 09:30:00,BSX,159020,2024-10-01 09:30:00,83.411003,83.680000,83.860001,84.000000
79,2024-10-01 09:35:00,BSX,87146,2024-10-01 09:35:00,83.614998,84.209999,83.620003,84.209999
80,2024-10-01 09:40:00,BSX,48543,2024-10-01 09:40:00,84.070000,84.129997,84.125000,84.250000
81,2024-10-01 09:45:00,BSX,44714,2024-10-01 09:45:00,83.884666,83.964996,84.099998,84.099998
82,2024-10-01 09:50:00,BSX,44518,2024-10-01 09:50:00,83.860001,83.864998,83.949997,84.059998


In [10]:
candle = {}
last_processed_index = {}
pointer_a = 0
pointer_b = 1
alert_data = {}
last_processed_interval = 0

def static_process(data):
    global candle, last_processed_index, pointer_a, pointer_b, alert_data, last_processed_interval

    # Store the processed index in a dict
    if data['symbol'] not in last_processed_index:
        last_processed_index[data['symbol']] = -1
            
    # Store the candle data in a dict
    if data['symbol'] not in candle:
        candle[data['symbol']] = []
        
    # Initialize alert_data for the symbol if it does not exist
    if data['symbol'] not in alert_data:
        alert_data[data['symbol']] = []
    
    # Append the data to the candle list
    candle[data['symbol']].append(data)
    
    # Loop through the candle data from the last processed index
    for i in range(last_processed_index[data['symbol']] + 1, len(candle[data['symbol']])):
        if last_processed_index[data['symbol']] == -1:
            pointer_a = 0
            pointer_b = 1
            curr_candle = candle[data['symbol']][pointer_a]
            candle_pattern = CandlePattern(curr_candle)
            bullish_382, bullish_382_date = candle_pattern.hammer_alert()
            if bullish_382:
                alert_data[data['symbol']].append({'date': bullish_382_date, 'alert': 'bullish_382'})
                hammer_count += 1
            
        curr_candle = candle[data['symbol']][i]                                                             
        candle_pattern = CandlePattern(curr_candle)
        
        # Check if there are enough candles to compare
        if len(candle[data['symbol']]) > 1 and pointer_b < len(candle[data['symbol']]):
            pre_candle = candle[data['symbol']][pointer_a]
            curr_candle = candle[data['symbol']][pointer_b]
            print(f"pre_candle: {pre_candle['datetime']} | curr_candle: {curr_candle['datetime']}")
            print(f"current candle: {curr_candle['datetime']}")
            # Check for bullish 0.382 candle patterns
            bullish_382, bullish_382_date = candle_pattern.hammer_alert()

            # Check for engulfing patterns
            bullish_engulfing, bearish_engulfing, date = candle_pattern.engulf_alert(pre_candle, curr_candle)

            if bullish_engulfing:
                alert_data[data['symbol']].append({'date': date, 'alert': 'bullish_engulfing'})
            elif bearish_engulfing:
                alert_data[data['symbol']].append({'date': date, 'alert': 'bearish_engulfing'})
            elif bullish_382:
                alert_data[data['symbol']].append({'date': bullish_382_date, 'alert': 'bullish_382'})
                
            # Update the last processed index
            last_processed_index[data['symbol']] = pointer_b
            print(f"symbol: {data['symbol']} comparing {pre_candle['datetime']} and {curr_candle['datetime']} | Processed Index: {last_processed_index[data['symbol']]}")
            print(f"pointer a: {pointer_a} | pointer b: {pointer_b}")
            print(f"current candle: {curr_candle['datetime']}")

        
        pointer_a += 1
        pointer_b += 1                

        break
    
# Test the function with the generated test data
for index, row in df.iterrows():
    static_process(row)

pre_candle: 2024-10-01 09:30:00 | curr_candle: 2024-10-01 09:35:00
current candle: 2024-10-01 09:35:00
symbol: BSX comparing 2024-10-01 09:30:00 and 2024-10-01 09:35:00 | Processed Index: 1
pointer a: 0 | pointer b: 1
current candle: 2024-10-01 09:35:00
pre_candle: 2024-10-01 09:35:00 | curr_candle: 2024-10-01 09:40:00
current candle: 2024-10-01 09:40:00
symbol: BSX comparing 2024-10-01 09:35:00 and 2024-10-01 09:40:00 | Processed Index: 2
pointer a: 1 | pointer b: 2
current candle: 2024-10-01 09:40:00
pre_candle: 2024-10-01 09:40:00 | curr_candle: 2024-10-01 09:45:00
current candle: 2024-10-01 09:45:00
symbol: BSX comparing 2024-10-01 09:40:00 and 2024-10-01 09:45:00 | Processed Index: 3
pointer a: 2 | pointer b: 3
current candle: 2024-10-01 09:45:00
pre_candle: 2024-10-01 09:45:00 | curr_candle: 2024-10-01 09:50:00
current candle: 2024-10-01 09:50:00
symbol: BSX comparing 2024-10-01 09:45:00 and 2024-10-01 09:50:00 | Processed Index: 4
pointer a: 3 | pointer b: 4
current candle: 2024

### Visualize the Candlesticks Pattern

In [11]:
# Select the data for the specific symbol (example: 'BSX')
symbol = 'BSX'
select_df = df[df['symbol'] == symbol]

# Create subplots: 2 rows, 1 column
fig = make_subplots(subplot_titles=('Candlestick Chart', 'Candle Pattern Alerts'))

# Add candlestick chart to the first subplot
fig.add_trace(go.Candlestick(x=select_df['datetime'].astype(str),  # Convert to string for categorical x-axis
                            open=select_df['open'],
                            high=select_df['high'],
                            low=select_df['low'],
                            close=select_df['close'],
                            increasing_line_color='green',  # Color for bullish (increasing) candles
                            decreasing_line_color='red'  # Color for bearish (decreasing) candles
                        ), row=1, col=1)

# Add candle pattern alerts to the candlestick chart
if symbol in alert_data:
    for alert in alert_data[symbol]:
        color = 'blue' if 'bullish' in alert['alert'] else 'red'
        fig.add_trace(go.Scatter(x=[alert['date']], 
                                y=[select_df[select_df['datetime'] == alert['date']]['close'].values[0]], 
                                mode='markers', 
                                marker=dict(color=color, size=10), 
                                name=alert['alert']), row=1, col=1)

# Update layout
fig.update_layout(
    title=f'Candlestick chart and Candle Pattern Alerts for {symbol}',
    xaxis_rangeslider_visible=False,  # Hide range slider
    showlegend=False,  # Show legend    
    xaxis_type='category'  # Set x-axis type to categorical to remove gaps
)

# Show the figure
fig.show()